$\textbf{Intro}$

In this work we are going to build a regression-based momentum alpha factor.

The term regression-based momentum alpha factor refers to a kind of alpha factor that combine linear regressions and simple returns. That means, a first alpha factor is constructed as a simple stock returns within a considered time window; the main time winow. In addition to that alpha factor, a second alpha factor is systematically constructed as the slop of a linear regression model built from a smaller sub time window of the main time window.

The smaller time window covers periods ranging from a certain time point within the main time window until the upper bound of the latter. In this way, we believe that combining the slop and the returns yield strong information on stock price's futur direction. For instance, if the slop is strongly positive and the return is strongly positive, then we hope to see a strong upward trend and vice versa. Hence, long and short positions will be taken accordingly. More details on the strategy will be shown later. 

The trading strategy will be evaluated using metrics that include, expected forward returns, annualized Sharpe Ratio, information coefficient, Turnover, etc.


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import alphalens as al
from sklearn.linear_modelo import LinearRegression
from alphalens.plotting import*
from alphalens.tears import*

ModuleNotFoundError: No module named 'sklearn.linear_modelo'